# Imports

In [1]:
from meteostat import Point, Daily, Hourly

import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import holidays
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# Traffic data

In [2]:
def extract_traffic():
    sts_df=pd.read_csv('sts.csv',sep=';',parse_dates=['Date debut dispo data','Date et heure de comptage','Date fin dispo data'])
    washington_df=pd.read_csv('washington.csv',sep=';',parse_dates=['Date debut dispo data','Date et heure de comptage','Date fin dispo data'])
    convention_df=pd.read_csv('convention.csv',sep=';',parse_dates=['Date debut dispo data','Date et heure de comptage','Date fin dispo data'])

    data_traffic=pd.concat([sts_df,washington_df,convention_df],axis=0)

    data_traffic['Date et heure de comptage']=pd.to_datetime(data_traffic['Date et heure de comptage'], utc=True)
    data_traffic['Date et heure de comptage']=data_traffic['Date et heure de comptage'].dt.strftime('%Y-%m-%d %X') 
    data_traffic['Date et heure de comptage']=pd.to_datetime(data_traffic['Date et heure de comptage'])

    data_traffic.drop(columns=[
    'Identifiant noeud amont', 'Identifiant noeud aval', 'geo_point_2d',
    'geo_shape', 'Date debut dispo data', 'Date fin dispo data','Libelle noeud aval','Libelle noeud amont','Identifiant arc', 'Etat arc'],inplace=True,axis=1)
    
    return data_traffic

extract_traffic()

,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic
0,Sts_Peres,2022-01-02 21:00:00,345.0,3.85500,Fluide
1,Sts_Peres,2022-01-02 19:00:00,482.0,5.92944,Fluide
2,Sts_Peres,2022-01-02 22:00:00,260.0,3.10555,Fluide
3,Sts_Peres,2022-01-02 16:00:00,523.0,10.19222,Fluide
4,Sts_Peres,2022-01-02 12:00:00,422.0,5.34889,Fluide
...,...,...,...,...,...
8405,Convention,2022-10-31 13:00:00,685.0,3.70889,Fluide
8406,Convention,2022-08-01 01:00:00,140.0,0.68445,Fluide
8407,Convention,2022-08-01 00:00:00,226.0,1.09111,Fluide
8408,Convention,2022-07-31 23:00:00,267.0,1.47611,Fluide


# Meteorological data

In [3]:
def extract_meteo(start, end):
    """
    Extract meteo data from Meteostat API
    start: start date
    end: end date
    """
    # Create Point for Paris, BC
    location = Point(48.856614, 2.333333)

    data_=Hourly(location, start, end)
    data_=data_.fetch()
    #using reset_index() to set index into column
    data_=data_.reset_index()
    data_.rename( {'time':'date','temp':'temperature', 'prcp':'total_precipitation_in_mm','snow':'snow_depth','tsun':'sunshine_minutes'}, axis=1, inplace=True)
    data_=data_[['date', 'temperature', 'total_precipitation_in_mm']]

    data_['date']=pd.to_datetime(data_['date'])

    return data_

# Full dataset

In [4]:
def create_dataset():
    """
    Create full dataset
    """
    data_traffic=extract_traffic()
    # extract traffic min date and max date
    min_date=data_traffic['Date et heure de comptage'].min()
    max_date=data_traffic['Date et heure de comptage'].max()

    # extract meteo data
    meteo=extract_meteo(min_date,max_date)

    data=data_traffic.merge(meteo,left_on='Date et heure de comptage',right_on='date',how='left')
    data.drop(columns=['date'],inplace=True,axis=1)

    return data

# data = create_dataset()
# data

# Data engineering

In [5]:
def engineer(data):
    """
    Engineer features
    """
    data_traffic=data.copy()

    # Date
    data_traffic['Date']=data_traffic['Date et heure de comptage'].dt.strftime('%Y-%m-%d')
    data_traffic['Date']=pd.to_datetime(data_traffic['Date'])
    # Time, weekday
    data_traffic['Time']=data_traffic['Date et heure de comptage'].dt.strftime('%H').astype(int)
    data_traffic['weekday']=data_traffic['Date et heure de comptage'].dt.weekday # Monday=0, Sunday=6
    # Holiday
    holidays_list=holidays.France(years=[2021, 2022]).keys() # this is a dict
    data_traffic.loc[data_traffic['Date'].isin(holidays_list),'holiday']=1
    data_traffic['holiday'].fillna(0,inplace=True)
    data_traffic.drop(columns=['Date'],inplace=True,axis=1)

    return data_traffic

# data_engineered=engineer(data)
# data_engineered

# Preprocessing

In [6]:
def encode(data):
    """
    Encode data
    """
    data_traffic=data.copy()
    
    # 'Etat trafic'
    mapper = {'Inconnu': 0, 'Fluide': 1, 'Pré-saturé': 2, 'Saturé': 3, 'Bloqué': 4}
    data_traffic['Etat trafic'] = data_traffic['Etat trafic'].map(mapper)

    # 'weekday', 'Libelle'
    encoder = OneHotEncoder()
    encoder_df = pd.DataFrame(encoder.fit_transform(data_traffic[['weekday','Libelle']]).toarray(),columns=encoder.get_feature_names_out())
    data_traffic = data_traffic.join(encoder_df)

    # 'Time'
    for i in range(0,24):
        data_traffic['time_'+str(i)] = np.where(data_traffic['Time']==i, 1, 0)

    # drop columns
    data_traffic.drop(columns=['weekday','Libelle','Time'],inplace=True,axis=1)

    return data_traffic


def preprocess(data, to_encode=True, index=True):
    """
    Preprocess data & Remove Datetime column
    """
    data_traffic=data.copy()

    if to_encode:
        # Encoded data
        data_traffic=encode(data_traffic)

    # Fill Misssing values by linear interpolation
    data_traffic['Débit horaire']=data_traffic['Débit horaire'].interpolate(method='linear')
    data_traffic["Taux d'occupation"]=data_traffic["Taux d'occupation"].interpolate(method='linear')

    if index:
        # Set index
        data_traffic.set_index('Date et heure de comptage',inplace=True)

    return data_traffic

# data_preprocessed=preprocess(data_engineered, to_encode=False, index=False)
# data_preprocessed

# Model 0 : Baseline model (simple moving average)

In [7]:
# def baseline(df):
#     ## Median value on last 5 weeks
#     max_date = df_limit.date_utc.max()
#     min_date = max_date - timedelta(days = nb_weeks*7)
#     #print("max ", max_date)
#     #print("min ", min_date)
#     df_filtered = df_limit[(df_limit.date_utc>min_date) & (df_limit.date_utc<=max_date)]
#     #print(df_filtered.weekday_utc.value_counts())
#     feature_debit = df_filtered[["time_utc", "weekday_utc", "Débit horaire"]].groupby(["weekday_utc", "time_utc"]).median()
#     feature_taux = df_filtered[["time_utc", "weekday_utc", "Taux d'occupation"]].groupby(["weekday_utc", "time_utc"]).median()
#     ## Merge the two features 
#     features = pd.merge(feature_debit, feature_taux,  how='left', on=['weekday_utc','time_utc'])

#     pred = pd.merge(pred, features,  how='left', on=['weekday_utc','time_utc'])

#     return pred

# Models

In [12]:
models = [
    # 'baseline',
    LinearRegression(),
    RandomForestRegressor(),
    xgb.XGBRegressor()
]

# Back Testing

In [27]:
def back_test(data, start_datetime, nb_rows, model):
    """
    Back test
    """
    # Split train & test
    train = data[(data.index < start_datetime)]
    test = data[(data.index >= start_datetime) & (data.index < start_datetime + timedelta(hours=nb_rows))]
    
    y_train = train[['Débit horaire', "Taux d'occupation"]]
    X_train = train.drop(columns=['Débit horaire', "Taux d'occupation"])
    
    y_test = test[['Débit horaire', "Taux d'occupation"]]
    X_test = test.drop(columns=['Débit horaire', "Taux d'occupation"])

    if model == 'baseline':
        y_pred = baseline(X_test)

    else:
        # Fit model
        model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)

    # Evaluate
    mape_debit = mean_absolute_percentage_error(y_test['Débit horaire'], y_pred[:,0])
    rmse_debit = mean_squared_error(y_test['Débit horaire'], y_pred[:,0], squared=False)
    mape_debit = round(mape_debit, 2)
    rmse_debit = round(rmse_debit, 2)
    print("MAPE Débit horaire: %s" % (100*mape_debit), '%')
    print("RMSE Débit horaire: ", rmse_debit)

    mape_taux = mean_absolute_percentage_error(y_test["Taux d'occupation"], y_pred[:,1])
    rmse_taux = mean_squared_error(y_test["Taux d'occupation"], y_pred[:,1], squared=False)
    mape_taux = round(mape_taux, 2)
    rmse_taux = round(rmse_taux, 2)
    print("MAPE Taux d'occupation: %s" % (100*mape_taux), '%')
    print("RMSE Taux d'occupation: ", rmse_taux)

    df_pred = pd.DataFrame(y_pred, columns=['Débit horaire', "Taux d'occupation"], index=y_test.index)
    df_pred = df_pred.reset_index()
    
    return df_pred

# Results : RMSE & MAPE

In [29]:
def results(start_datetime, model, nb_rows=120):
    data = create_dataset()
    data_engineered=engineer(data)
    data_preprocessed=preprocess(data_engineered, to_encode=False, index=False)
    data_preprocessed_index=preprocess(data_engineered, to_encode=False, index=True)
    data_preprocessed_encoded=preprocess(data_engineered, to_encode=True, index=True)

    df_pred = back_test(data_preprocessed_encoded, start_datetime, nb_rows, model)

    # add Libelle column
    df_pred = df_pred.merge(data_preprocessed[['Libelle', 'Date et heure de comptage']],  how='left', left_on='Date et heure de comptage', right_on='Date et heure de comptage')

    df_pred.sort_values(by='Date et heure de comptage', inplace=True)

    # Plot
    df_pred_libelle = df_pred[df_pred['Libelle'] == 'Convention'].set_index('Date et heure de comptage')

    # df_pred_libelle['Débit horaire'].plot(figsize=(20,5))
    # plt.show()
    # df_pred_libelle['Taux d\'occupation'].plot(figsize=(20,5))
    # plt.show()

    return df_pred

In [30]:
for model in models:
    print('')
    print(model)
    results(datetime(2022, 11, 25, 18, 0, 0), model)


LinearRegression()
MAPE Débit horaire: 45.0 %
RMSE Débit horaire:  225.47
MAPE Taux d'occupation: 50.0 %
RMSE Taux d'occupation:  3.5

RandomForestRegressor()
MAPE Débit horaire: 40.0 %
RMSE Débit horaire:  223.5
MAPE Taux d'occupation: 20.0 %
RMSE Taux d'occupation:  2.49

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)


# Prediction

In [ ]:
df_pred = results(datetime(2022, 12, 9, 18, 0, 0), model = models[-1])

df_pred.to_csv('output_MAIDY.csv')

IndexError: index 0 is out of bounds for axis 1 with size 0